In [1]:
import json
from datatypes.vessel import Vessel
from datatypes.track import Track
from dynamicSceneGenerator import DynamicSceneGenerator

from visualize import visualize_dynamic_scene, visualize_camera_pose_in_dsg, visualize_projections, visualize_bounding_boxes
from MODSIM import create_and_place_simple_legacy_camera, project_all_points, create_all_bbs, tracks_to_json, vessels_to_json


In [2]:
# Get vessels

with open('./henriksPoseData/vehicle_characteristics_henrik.json', 'r') as f:
    vessel_dict = json.load(f)

# air_draft: int = 2, beam: int = 2, length: int = 4, label: str = "")
vessels = [Vessel(vesselID, length=vessel['length_m'], beam=vessel['width_m'], label=vessel['type']) for vesselID, vessel in vessel_dict.items()]

In [3]:
# Create dynamic scene with vessels

dsg = DynamicSceneGenerator(vessels)

In [4]:
# Load tracks 
with open('./henriksPoseData/dynamics.json', 'r') as f:
    dynamics = json.load(f)

tracks = {vessel.id: Track() for vessel in vessels}

for t, positions in dynamics['time_s'].items():
    for vesselID, position in positions.items():
        x = position['center_position_m'][0]
        y = position['center_position_m'][1]
        z = 0
        heading_rad = position['heading_rad'] 
        time_stamp = float(t)
        tracks[vesselID].addPosition(x, y, z, heading_rad, time_stamp)

In [5]:
# Update each vessel in the scene with the correct track
for vessel in dsg.get_vessels():
    vessel.set_track(tracks[vessel.id])

In [6]:
simulation_folder = './henriksPoseData/'

vessels = dsg.get_vessels()
tracks_to_json(vessels, simulation_folder)
vessels_to_json(vessels, simulation_folder)

In [7]:
# visualize_dynamic_scene(vessels, folder_path=simulation_folder, y_x_lim=150)

In [8]:
largest_radius = 75
path_centre = (75,40)
height = 60
camera = create_and_place_simple_legacy_camera(largest_radius, path_centre, height=height)

In [9]:
# visualize_camera_pose_in_dsg(camera, vessels, folder_path=simulation_folder)

In [10]:
all_projected_points = project_all_points(camera, vessels, writeToJson=True, folder_path=simulation_folder)

In [11]:
# visualize_projections(all_projected_points, camera.image_bounds, folder_path=simulation_folder)

In [12]:
all_bbs=create_all_bbs(all_projected_points, writeToJson=True, folder_path=simulation_folder)

In [13]:
# visualize_bounding_boxes(all_bbs, camera.image_bounds, folder_path=simulation_folder)